<a href="https://colab.research.google.com/github/elemi10/dataAnalysis-Porforlio/blob/gh-pages/Trabajo_Pr%C3%A1ctico_TIAAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Establecemos constantes generales

In [ ]:
# Nombres de las columnas por orden, según indica la planilla
HEADERS = ["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "modelyear", "origin", "carname"]

# Columnas con valores discretos según planilla
VARIABLES_DISCRETAS = ['cylinders', 'origin', 'modelyear']

Cargamos el archivo ya almacenado en Colab

In [ ]:
# Cargamos archivo especificando los headers
autos = pd.read_excel('/content/sample_data/DatasetAutomoviles.xls', names = HEADERS)

In [ ]:
autos.describe

Rmovemos aquellas filas que tienen valores incompletos

In [ ]:
print('Todos los valores:', len(autos))
autos = autos[autos.horsepower != '?']
print('Sin valores incompletos:', len(autos))

# ¿En vez de eliminar los valores incompletos quizás se podrían reemplazar por 0?
# Reemplazamos los valores faltantes '?' de "horsepower" por 0
# autos['horsepower'].replace(r'\?', 0, regex = True, inplace = True)

Todos los valores: 397
Sin valores incompletos: 391


*Sanitizamos* y homogeneizamos valores

In [ ]:
# Limpiamos tabulaciones extras que aparecen en la columna "carname"
autos['carname'].replace(r'\t', '', regex = True, inplace = True)

# Convesión de dtypes para eliminar diferencias en los decimales, puntos y ceros innecesarios
autos['mpg'] = autos['mpg'].astype('int64')
autos['displacement'] = autos['displacement'].astype('int64')
autos['horsepower'] = autos['horsepower'].astype('float64').astype('int64') # primero se pasa a float y luego a int para evitar error en la conversión
autos['weight'] = autos['weight'].astype('int64')

Creamos variables dummy para todas las columnas con valores discretos

In [ ]:
# Esto automáticamente elimina los valores originales de esas columnas
autos = pd.get_dummies(autos, columns = VARIABLES_DISCRETAS, prefix_sep = "_")

# Revisamos las columnas que tenemos hasta ahora
print(autos.columns)

Index(['mpg', 'displacement', 'horsepower', 'weight', 'acceleration',
       'carname', 'cylinders_3', 'cylinders_4', 'cylinders_5', 'cylinders_6',
       'cylinders_8', 'origin_1', 'origin_2', 'origin_3', 'modelyear_70',
       'modelyear_71', 'modelyear_72', 'modelyear_73', 'modelyear_74',
       'modelyear_75', 'modelyear_76', 'modelyear_77', 'modelyear_78',
       'modelyear_79', 'modelyear_80', 'modelyear_81', 'modelyear_82'],
      dtype='object')


In [ ]:
# Print de todos los valores para ver cómo vienen los datos
print(autos.to_string())

Correlaciones

In [ ]:
correlaciones = autos.corr(method = 'pearson')['mpg']
print(correlaciones.sort_values(ascending = False))

mpg             1.000000
cylinders_4     0.764591
origin_3        0.451360
acceleration    0.422665
modelyear_80    0.355714
modelyear_82    0.324101
origin_2        0.247391
modelyear_81    0.232806
modelyear_79    0.048409
cylinders_5     0.042065
modelyear_78    0.015220
modelyear_77   -0.008081
modelyear_74   -0.018185
cylinders_3    -0.040012
modelyear_76   -0.075142
modelyear_71   -0.076892
modelyear_75   -0.112834
modelyear_72   -0.164401
modelyear_70   -0.201559
cylinders_6    -0.235611
modelyear_73   -0.270223
origin_1       -0.567754
cylinders_8    -0.650255
horsepower     -0.779159
displacement   -0.806562
weight         -0.834905
Name: mpg, dtype: float64


Creamos los dataframes para los modelos predictivos


In [ ]:
y = autos['mpg'] # variable dependienente
x = autos.drop(columns = ['mpg', 'carname'], axis = 1) # variables independenientes

Normalizamos los valores

In [ ]:
# "A los valores de la variable, restarle la media y dividirla por el desvío estandar"
x_norm = ( x - x.mean() ) / x.std()
print(x_norm)

     displacement  horsepower  ...  modelyear_81  modelyear_82
0        1.489885    1.573154  ...     -0.277376     -0.287906
1        1.184027    1.183733  ...     -0.277376     -0.287906
2        1.050214    1.183733  ...     -0.277376     -0.287906
3        1.031098    0.924119  ...     -0.277376     -0.287906
4        2.244973    2.429880  ...     -0.277376     -0.287906
..            ...         ...  ...           ...           ...
392     -0.517309   -0.477796  ...     -0.277376      3.464471
393     -0.928306   -1.360483  ...     -0.277376      3.464471
394     -0.565099   -0.529719  ...     -0.277376      3.464471
395     -0.708470   -0.659526  ...     -0.277376      3.464471
396     -0.718029   -0.581641  ...     -0.277376      3.464471

[391 rows x 25 columns]


Predicciones

In [ ]:
# Divididmos la data de modo random para testear y entrenar 
x_train, x_valid, y_train, y_valid = train_test_split(x_norm, y, test_size = 0.1)

# Clasificador
model1 = LogisticRegression(solver = 'liblinear', random_state = 0, verbose = 1, C = 1.0, penalty = 'l2')
model1.fit(x_train, y_train)

coef1 = pd.DataFrame(model1.coef_).T
coef1['col'] = pd.DataFrame(x_train.columns)

y_pred1 = pd.DataFrame(model1.predict_proba(x_train), index=x_train.index )
y_pred1['pct_rank'] = np.round(y_pred1[0].rank(pct=True)*10,0)

y_pred1b = y_pred1b.reset_index()


[LibLinear]